In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

from grav_lens.models.load_models import load_minmaxscaler, load_ipca_low
# Cargar el MinMaxScaler
minmaxscaler = load_minmaxscaler()
print(minmaxscaler)
# Cargar el modelo IPCA low
ipca_low = load_ipca_low()
print(ipca_low)


from grav_lens import get_datasets
max_files = 2000
batch_size = 64
home_data = os.path.join("..","data")

train_dataset, val_dataset, test_dataset = get_datasets(
    data_index=1,
    max_files=max_files,
    home=home_data,
    batch_size=batch_size,
    val_split=0.2,
    test_split=0.1,
)

for X_batch, y_batch in train_dataset.take(1):
    print(X_batch.shape)
    print(y_batch.shape)



IncrementalPCA(batch_size=64, n_components=64)
Using data folder: ..\data\1
(64, 128, 128, 3)
(64, 128, 128, 1)


In [20]:
import grav_lens.preprocess as gp
from grav_lens.utils.statistics import get_stats
from grav_lens.preprocess.filtering import process_batch_filters

## Mini tutorial de las funciones más usadas
Algunas funciones de utilidad
- gp.apply_threshold
- gp.gmm_batch_vectors

In [18]:
# se puede separar por encima o abjao de un punto de corte
batch_positive, batch_negative =   gp.apply_threshold(y_batch, *get_stats(y_batch))

In [21]:
# separa en baja y alta frecuencia 5.2 s por batch
low_batch, high_batch = process_batch_filters(y_batch)

- Los componentes de baja frecuencia son bien aproximados por componentes principales
- Los componentes de alta frecuencia son bien aproximados por peaks gaussianos

In [24]:
gp.gmm_batch_vectors?

Signature:
gp.gmm_batch_vectors(
    batch,
    n_gaussians_positive=30,
    n_gaussians_negative=10,
    threshold=2,
)
Docstring:
Aplica modelos de mezclas gaussianas (GMM) a un batch de imágenes, generando vectores de medias, desviaciones estándar y pesos para cada imagen.

Parámetros:
    batch (numpy array): Batch de imágenes a procesar (batch_size, altura, anchura, 1).
    n_gaussians_positive (int, opcional): Número de componentes gaussianas positivas (por defecto 30).
    n_gaussians_negative (int, opcional): Número de componentes gaussianas negativas (por defecto 10).
    threshold (float, opcional): Umbral para separar las frecuencias (por defecto 2).

Retorna:
    combined_batch (numpy array): Batch de vectores combinados de medias, desviaciones estándar y pesos de cada imagen,
                                  con shape (batch_size, n_gaussianas, 5). El vector tiene el formato 
                                  [mean_x, mean_y, std_x, std_y, weight] para cada gaussiana.
Fil

In [26]:
# 15 s por batch
# (batch, 40, 5) 
# 40 es el total de gaussianas
# 5 el vector correspondiente a cada gaussiana
gaussians = gp.gmm_batch_vectors(high_batch)

(64, 40, 5)

# El proceso completo
De manera que tenemos las dos formas principales de datos
- gaussians (batch, 40, 5), con 40 el total de gaussianas, 5 el largo del vector de info: 
    meanx, meany, stdx, stdy, coef
- principal_components (batch, 64), con 64 coeficientes que representan cuanto hay de cada componente


In [27]:
for X_batch, y_batch in train_dataset.take(1):
    y_batch = minmaxscaler.transform(y_batch) # [0, 1] #escalar los datos

    print(X_batch.shape)
    print(y_batch.shape)
    
    low_batch, high_batch = process_batch_filters(y_batch)

    gaussians = gp.gmm_batch_vectors(high_batch)

    low_freq_stack = np.vstack([img.reshape(-1, 128*128) for img in low_batch]) #stack para el pca
    principal_components = ipca_low.transform(low_freq_stack)


    

(64, 128, 128, 3)
(64, 128, 128, 1)
